## Problem Statement


In today's digital age, social media platforms are very much popular amongst all age categories. Approximately, 63.9% of the words population (5.24 billion) uses social media platforms with an average user spending roughly 2 hours 21 minutes on it daily. Today, social media platforms have become the primary channel for communication and interactions. However, since 2013 there has been a steady rise on the overall negativity present on the platforms. The main driving force for this rise is due to the hate speech and offensive language remarks present all over social media. These forms of expression can perpetuate discrimination, foster hostility, and negatively impact ones mental well-being.

Hate speech is generally defined as any communication that attacks or denigrates a person or group based on attributes like race, religion, ethnicity, gender, sexual orientation, or disability. Offensive language, less severe, is defined by words or phrases that can be used in a disrespectful, insulting, or hurtful manner without targeting a particular group. Both are considered as a negative form of message to the opposite party.

The presence of such language leads to several significant issues:

User Safety: Hate speech and offensive language create unsafe online spaces, discouraging healthy discussions and making users feel alienated.
Platform Reputation: Platforms that fail to address these concerns risk losing user trust and facing scrutiny from regulators.
Legal and Ethical Implications: Many countries enforce strict laws against hate speech, requiring platforms to take proactive measures to filter such content.
Scalability of Moderation: Given the vast amount of content generated daily, manual moderation is neither practical nor scalable.
Our goal is to build a model which can identify such negative remarks and comments and prevent it from reaching the victim before its too late. This way we can make social media a safe environment for everyone to use and foster healthy relationships between the users.

![Image Description](image.webp)

## Background Information on the dataset

The dataset we are using, named hate_speech_offensive, is a meticulously curated collection of annotated tweets with the specific purpose of detecting hate speech and offensive language. The dataset primarily consists of English tweets and is designed to train machine learning models or algorithms in the task of hate speech detection. It should be noted that the dataset has not been divided into multiple subsets, and only the train split is currently available for use.

The dataset includes several columns that provide valuable information for understanding each tweet's classification. The column count represents the total number of annotations provided for each tweet, whereas hate_speech_count signifies how many annotations classified a particular tweet as hate speech. On the other hand, offensive_language_count indicates the number of annotations categorizing a tweet as containing offensive language. Additionally, neither_count denotes how many annotations identified a tweet as neither hate speech nor offensive language.

For researchers and developers aiming to create effective models or algorithms capable of detecting hate speech and offensive language on Twitter, this comprehensive dataset offers a rich resource for training and evaluation purposes

It consists of annotated tweets with information about their classification as hate speech, offensive language, or neither. Each row represents a tweet along with the corresponding annotations provided by multiple annotators. The main columns that will be essential for your analysis are: count (total number of annotations), hate_speech_count (number of annotations classifying a tweet as hate speech), offensive_language_count (number of annotations classifying a tweet as offensive language), neither_count (number of annotations classifying a tweet as neither hate speech nor offensive language).

The data collection methodology used to create this dataset involved obtaining tweets from Twitter's public API using specific search terms related to hate speech and offensive language. These tweets were then manually labeled by multiple annotators who reviewed them for classification purposes.

In [42]:
# Machine Learning Packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight

# Natural Language Processing (NLP) Libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import string

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
from lime.lime_text import LimeTextExplainer
from collections import OrderedDict

# Data Manipulation and Processing
import pandas as pd
import numpy as np

# Imbalanced Data Handling
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [43]:
df=pd.read_csv("train.csv")

In [44]:
df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   count                     24783 non-null  int64 
 1   hate_speech_count         24783 non-null  int64 
 2   offensive_language_count  24783 non-null  int64 
 3   neither_count             24783 non-null  int64 
 4   class                     24783 non-null  int64 
 5   tweet                     24783 non-null  object
dtypes: int64(5), object(1)
memory usage: 1.1+ MB


In [46]:
df.shape

(24783, 6)

In [47]:
df.columns

Index(['count', 'hate_speech_count', 'offensive_language_count',
       'neither_count', 'class', 'tweet'],
      dtype='object')

In [48]:
df.describe()

,count,hate_speech_count,offensive_language_count,neither_count,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [49]:
df['class'].unique()

array([2, 1, 0])

In [50]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

Right now the target variable has 3 classes (Hate speech, Offensive Language, Neither). The dataset is dominated by offensive lanugage tweets.

## Baseline Modelling without any feature engineering/hyperparameter tuning

In [51]:
# features & labels
x=df['tweet']
y=df['class']
# Split the Dataset
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

### Count Vectorizer vs Tfidf Vectorizer

CountVectorizer does the job of creating a word count table. It takes in the collection of text data and converts it into a matrix of token counts. Each row represents the text/sentence and each column represents a unique word (or token). The values indicates how many times each word appears within the sentence/text.

Suppose you have three sentences:

"I love coding."
"Coding is fun."
"I love learning new things."
Using CountVectorizer, the result might look something like this:

|        | coding | fun | i | is | learning | love | new | things |
|--------|-------|-----|---|----|----------|------|-----|-------|
| Doc 1  | 1     | 0   | 1 | 0  | 0        | 1    | 0   | 0     |
| Doc 2  | 1     | 1   | 0 | 1  | 0        | 0    | 0   | 0     |
| Doc 3  | 0     | 0   | 1 | 0  | 1        | 1    | 1   | 1     |

TfidfVectorizer is an extension of CountVectorizer. While CountVectorizer counts the words, TfidfVectorizer goes a step further and addtionally considers the importance of words across all the sentences. It assigns more weight to words that appear more frequently in a single input but are rare across other inputs, making it to better distinguish between words like "a" and actual meaningful terms.

|        | coding | fun | i | is | learning | love | new | things |
|--------|-------|-----|---|----|----------|------|-----|-------|
| Doc 1  | 0.70710678     | 0   | 0.70710678 | 0  | 0        | 0.70710678   | 0   | 0     |
| Doc 2  | 0.4736296     | 0.4736296   | 0 | 0.40204024  | 0       | 0    | 0   | 0     |
| Doc 3  | 0     | 0   | 0.52863461 | 0  | 0.40204024        | 0.40204024    | 0.52863461   | 0.52863461     |

In [52]:
# Create a pipeline with Count Vectorizer and Multinomial Naive Bayes classifier
pipe_nb_cv = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('nb', MultinomialNB())
])
# Create a pipeline with Tfidf Vectorizer and Multinomial Naive Bayes classifier
pipe_nb_tfidf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])
# Create a pipeline with Count Vectorizer and Logistic Regression classifier 
pipe_lr_cv = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('lr', LogisticRegression())
])
# Create a pipeline with Tfidf Vectorizer and Logistic Regression classifier
pipe_lr_tfidf = Pipeline(steps=[
    ('tfidf',TfidfVectorizer()),
    ('lr', LogisticRegression())
])
# Create a pipeline with Count Vectorizer and RandomForest Classifier
pipe_rf_cv = Pipeline([
    ('cv', CountVectorizer()),
    ('rf', RandomForestClassifier())
])
# Create a pipeline with Tfidf Vectorizer and RandomForest Classifier
pipe_rf_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier())
])
# Create a pipeline with CountVectorizer and SVM
pipe_svm_cv = Pipeline([
    ('cv', CountVectorizer()),
    ('svm', SVC(kernel='linear', C=1))
])
# Create a pipelin with Tfidf Vectorizer and SVM
pipe_svm_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1))
])

### Naive Bayes

In [53]:
pipe_nb_cv.fit(x_train,y_train)
pipe_nb_cv.score(x_test,y_test)

0.8533387129312084

In [54]:
pipe_nb_tfidf.fit(x_train,y_train)
pipe_nb_tfidf.score(x_test,y_test)

0.7865644543070406

### Linear Regression

In [55]:
pipe_lr_cv.fit(x_train,y_train)
pipe_lr_cv.score(x_test,y_test)

0.8991325398426467

In [56]:
pipe_lr_tfidf.fit(x_train,y_train)
pipe_lr_tfidf.score(x_test,y_test)

0.8908614081097438

### Random Forest

In [57]:
pipe_rf_cv.fit(x_train,y_train)
pipe_rf_cv.score(x_test,y_test)

0.8710913859189026

In [58]:
pipe_rf_tfidf.fit(x_train,y_train)
pipe_rf_tfidf.score(x_test,y_test)

0.8595924954609643

### Support Vector Machine (SVM)

In [59]:
pipe_svm_cv.fit(x_train,y_train)
pipe_svm_cv.score(x_test,y_test)

0.8971151906394997

In [60]:
pipe_svm_tfidf.fit(x_train,y_train)
pipe_svm_tfidf.score(x_test,y_test)

0.9025620334879968

### Confusion Matrixes (For the different classification algorithms)